In [3]:

import pandas as pd
# load data 
X_train = pd.read_csv('../../data/representation.eng.train.csv').to_numpy()

y_train = pd.read_csv('../../data/true_labels.eng.train.csv')

X_test = pd.read_csv('../../data/representation.eng.testa.csv').to_numpy()
X_test2 = pd.read_csv('../../data/representation.eng.testb.csv').to_numpy()
y_test = pd.read_csv('../../data/true_labels.eng.testa.csv')
y_test2 = pd.read_csv('../../data/true_labels.eng.testb.csv')




## Logistic regression

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
# Create a logistic regression model for multiclass classification
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       I-LOC       0.88      0.81      0.84        78
      I-MISC       0.82      0.57      0.67        58
       I-ORG       0.77      0.79      0.78        84
       I-PER       0.95      0.96      0.96       110
           O       0.98      1.00      0.99      1669

    accuracy                           0.97      1999
   macro avg       0.88      0.82      0.85      1999
weighted avg       0.96      0.97      0.96      1999



## Neural network 

In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, TimeDistributed
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# load data 
X_train = pd.read_csv('../../data/representation.eng.train.csv').to_numpy()

y_train = pd.read_csv('../../data/true_labels.eng.train.csv')

X_test = pd.read_csv('../../data/representation.eng.testa.csv').to_numpy()
X_test2 = pd.read_csv('../../data/representation.eng.testb.csv').to_numpy()
y_test = pd.read_csv('../../data/true_labels.eng.testa.csv')
y_test2 = pd.read_csv('../../data/true_labels.eng.testb.csv')

# Encode labels and convert to one-hot vectors
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train.values.ravel())
y_test_encoded = encoder.transform(y_test.values.ravel())

num_classes = len(np.unique(y_train_encoded))
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes)

# Neural network configuration
input_dim = X_train.shape[1]
embedding_dim = 128
lstm_units = 64

# Build the neural network
model = Sequential([
    Embedding(input_dim=input_dim, output_dim=embedding_dim, input_length=input_dim),
    Bidirectional(LSTM(lstm_units, return_sequences=False)),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#summary 
model.summary()


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Train the model

history = model.fit(X_train, y_train_one_hot, epochs=2, batch_size=32, validation_data=(X_test, y_test_one_hot))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_one_hot)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 187s 597ms/step - accuracy: 0.8340 - loss: 0.6822 - val_accuracy: 0.8349 - val_loss: 0.6787
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 193s 617ms/step - accuracy: 0.8387 - loss: 0.6643 - val_accuracy: 0.8349 - val_loss: 0.6747
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 148ms/step - accuracy: 0.8351 - loss: 0.6798
Test Loss: 0.6746992468833923, Test Accuracy: 0.8349174857139587
